<a href="https://colab.research.google.com/github/pneuly/whisper-asr-colab/blob/main/whisper_asr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from google.colab import files

# @title 自動文字起こし（Wisper）{ display-mode: "form" }
# @markdown 以下の設定項目を入力しセルを実行（Ctrl+Enter）<font color="red">※設定項目の説明は下にあります</font>
# @markdown #<b>設定</b>
audio = 'https://www.youtube.com/watch?v=xAmEQOqtMvA'  # @param {type:"string"}
model_size = "large-v3" # @param ["large-v3", "large-v2", "large", "medium", "small", "base", "tiny"] {allow-input: true}
diarization = True  # @param {type:"boolean"}
password = None  # @param {type:"string"}
start_time = None  # @param {type:"string"}
end_time = None  # @param {type:"string"}
timestamp_offset = None  # @param {type:"string"}
initial_prompt = "定刻 なりましたので、 です。 ます。"  # @param {type:"string"}
realtime = False  # @param {type:"boolean"}
CHUNK_SIZE = 30
BATCH_SIZE = 1
HUGGING_FACE_TOKEN = ""

# @markdown ###<br/><b>〔設定の説明〕</b>
# @markdown <b>audiopath:</b> 文字起こしする音声ファイルの場所<br/>
# @markdown 　　Youtubeの場合： https://www.youtube.com/......<br/>
# @markdown 　　手動で音声をアップロードした場合： 230401_1010.mp3 など<br/>
# @markdown 　　（アップロード完了まで待って実行してください）<br/>
# @markdown 　　<font color="red">空欄の場合はファイルアップロードボタンが表示されます</font>
# @markdown <br/><b>model_size:</b> 音声認識のモデルサイズ（mediumにすると少し精度が落ちるが早い）
# @markdown <br/><b>diarization:</b> 発言者別の文字起こしファイルを作成するか（Falseにすると早い）
# @markdown #### <br/><b><font color= "blue">以下は必要な場合のみ設定</font></b>
# @markdown <b>password:</b> パスワードを指定（Webexなど）</b>
# @markdown <br/><b>start_time:</b> 開始時間 hh:mm:ss</b>（指定しない場合は最初から）
# @markdown <br/><b>end_time:</b> 終了時間 hh:mm:ss（指定しない場合は最後まで）
# @markdown <br/><b>timestamp_offset:</b> タイムスタンプを指定の時間だけずらす hh:mm:ss（Noneの場合はstart_timeと連動）
# @markdown <br/><b>initial_prompt:</b> キーワード（です。 ます。は句読点を付けるために入れています）
# @markdown <br/><b>reatime: </b><font color="red">ストリーミングをリアルタイムで文字起こしをする場合のみオンにしてください。</font>

# ----- main routine ------
if audio == "":
    audio = list(files.upload())[0]

!pip install --upgrade git+https://github.com/pneuly/whisper-asr-colab.git@develop -q
from whisper_asr_colab.worker import Worker  # noqa: E402

# dl audio if needed
worker = Worker(
    audio = audio,
    model_size = model_size,
    diarization = diarization,
    password = password,
    start_time = start_time,
    end_time = end_time,
    timestamp_offset = timestamp_offset,
    initial_prompt = initial_prompt,
    realtime = realtime,
    chunk_size = CHUNK_SIZE,
    batch_size = BATCH_SIZE,
    hugging_face_token = HUGGING_FACE_TOKEN
)

worker.run()

# gc GPU RAM
del worker
torch.cuda.empty_cache()
